Create LSTM model using static data

In [1]:
import pandas as pd
import numpy as np


raw = pd.read_csv("NIFTY-50.csv")

print(raw)
raw.describe()

            Date          Open          High           Low         Close  \
0     2013-01-02   5982.600098   6006.049805   5982.000000   5993.250000   
1     2013-01-03   6015.799805   6017.000000   5986.549805   6009.500000   
2     2013-01-04   6011.950195   6020.750000   5981.549805   6016.149902   
3     2013-01-07   6042.149902   6042.149902   5977.149902   5988.399902   
4     2013-01-08   5983.450195   6007.049805   5964.399902   6001.700195   
...          ...           ...           ...           ...           ...   
2709  2023-12-22  21295.849609  21390.500000  21232.449219  21349.400391   
2710  2023-12-26  21365.199219  21477.150391  21329.449219  21441.349609   
2711  2023-12-27  21497.650391  21675.750000  21495.800781  21654.750000   
2712  2023-12-28  21715.000000  21801.449219  21678.000000  21778.699219   
2713  2023-12-29  21737.650391  21770.300781  21676.900391  21731.400391   

         Adj Close    Volume  
0      5993.250000       0.0  
1      6009.500000       

,Open,High,Low,Close,Adj Close,Volume
count,2700.000000,2700.000000,2700.000000,2700.000000,2700.000000,2.700000e+03
mean,11364.825725,11418.140963,11292.118393,11356.717970,11356.717970,2.935654e+05
std,4159.661856,4173.421486,4139.054223,4158.403861,4158.403861,1.950139e+05
min,5233.450195,5317.700195,5118.850098,5285.000000,5285.000000,0.000000e+00
25%,8172.975098,8213.499756,8123.375122,8170.649902,8170.649902,1.669750e+05
50%,10488.925293,10537.100097,10431.799805,10479.250000,10479.250000,2.316500e+05
75%,14932.824951,14990.312744,14774.962891,14907.149903,14907.149903,3.460500e+05
max,21737.650391,21801.449219,21678.000000,21778.699219,21778.699219,1.811000e+06


Cleaning Data and Extracting usefull features

In [2]:
# dropping nan rows
X = raw['Close']
X = X.dropna().reset_index().drop('index', axis=1)

